## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Rikitea,PF,-23.1203,-134.9692,68.11,77,100,7.74,light rain
1,1,Kapaa,US,22.0752,-159.3190,82.72,74,20,13.80,few clouds
2,2,Cherskiy,RU,68.7500,161.3000,54.59,39,76,4.23,broken clouds
3,3,Shache,CN,38.4167,77.2406,72.86,45,26,7.81,scattered clouds
4,4,Ribeira Grande,PT,38.5167,-28.7000,73.80,78,20,12.66,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,1,Kapaa,US,22.0752,-159.3190,82.72,74,20,13.80,few clouds
6,6,Vaini,TO,-21.2000,-175.2000,77.16,88,20,6.91,few clouds
13,13,Mawlaik,MM,23.6333,94.4167,79.00,86,10,1.90,clear sky
14,14,Cartago,CO,4.7464,-75.9117,83.95,64,27,2.17,scattered clouds
15,15,Butaritari,KI,3.0707,172.7902,81.68,81,100,18.50,moderate rain
17,17,Jamestown,US,42.0970,-79.2353,80.53,40,0,6.91,clear sky
18,18,Ribeira Brava,PT,32.6500,-17.0667,75.00,82,16,5.37,few clouds
20,20,Albany,US,42.6001,-73.9662,85.17,37,7,2.44,clear sky
21,21,Biscarrosse,FR,44.3945,-1.1672,84.15,32,0,8.05,clear sky
26,26,Wad Madani,SD,14.4012,33.5199,82.85,60,80,13.33,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                242
City                   242
Country                242
Lat                    242
Lng                    242
Max Temp               242
Humidity               242
Cloudiness             242
Wind Speed             242
Weather Description    242
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Kapaa,US,82.72,few clouds,22.0752,-159.3190,
6,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,
13,Mawlaik,MM,79.00,clear sky,23.6333,94.4167,
14,Cartago,CO,83.95,scattered clouds,4.7464,-75.9117,
15,Butaritari,KI,81.68,moderate rain,3.0707,172.7902,
17,Jamestown,US,80.53,clear sky,42.0970,-79.2353,
18,Ribeira Brava,PT,75.00,few clouds,32.6500,-17.0667,
20,Albany,US,85.17,clear sky,42.6001,-73.9662,
21,Biscarrosse,FR,84.15,clear sky,44.3945,-1.1672,
26,Wad Madani,SD,82.85,broken clouds,14.4012,33.5199,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Kapaa,US,82.72,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
6,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,Keleti Beach Resort
13,Mawlaik,MM,79.00,clear sky,23.6333,94.4167,AKZ Guest House
14,Cartago,CO,83.95,scattered clouds,4.7464,-75.9117,sammpacale
15,Butaritari,KI,81.68,moderate rain,3.0707,172.7902,Isles Sunset Lodge
17,Jamestown,US,80.53,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
18,Ribeira Brava,PT,75.00,few clouds,32.6500,-17.0667,Hotel do Campo
20,Albany,US,85.17,clear sky,42.6001,-73.9662,
21,Biscarrosse,FR,84.15,clear sky,44.3945,-1.1672,Saint Hubert
26,Wad Madani,SD,82.85,broken clouds,14.4012,33.5199,فندق يوغسلافيا


In [10]:
# I noticed the above did not drop the rows with blank Hotel Name values!

import numpy as np

clean_hotel_df = clean_hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df.head(10)

# I found the code to change blank values to NaNs from 
# https://sparkbyexamples.com/pandas/pandas-replace-blank-values-with-nan/
# as recycling the code from our PyCitySchools challenge (using .loc) wasn't working with a blank value

# I've realized (a day has passed) that I was using incorrect bracketing on my .loc formula, but I think at this point
# I'll keep this new formula in place as I'm now on Deliverable 3 and have already made multiple Google API calls...
# Please let me know if this poses any issues! 


,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Kapaa,US,82.72,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
6,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,Keleti Beach Resort
13,Mawlaik,MM,79.00,clear sky,23.6333,94.4167,AKZ Guest House
14,Cartago,CO,83.95,scattered clouds,4.7464,-75.9117,sammpacale
15,Butaritari,KI,81.68,moderate rain,3.0707,172.7902,Isles Sunset Lodge
17,Jamestown,US,80.53,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
18,Ribeira Brava,PT,75.00,few clouds,32.6500,-17.0667,Hotel do Campo
20,Albany,US,85.17,clear sky,42.6001,-73.9662,NaN
21,Biscarrosse,FR,84.15,clear sky,44.3945,-1.1672,Saint Hubert
26,Wad Madani,SD,82.85,broken clouds,14.4012,33.5199,فندق يوغسلافيا


In [11]:
clean_hotel_df = clean_hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Kapaa,US,82.72,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
6,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,Keleti Beach Resort
13,Mawlaik,MM,79.00,clear sky,23.6333,94.4167,AKZ Guest House
14,Cartago,CO,83.95,scattered clouds,4.7464,-75.9117,sammpacale
15,Butaritari,KI,81.68,moderate rain,3.0707,172.7902,Isles Sunset Lodge
17,Jamestown,US,80.53,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
18,Ribeira Brava,PT,75.00,few clouds,32.6500,-17.0667,Hotel do Campo
21,Biscarrosse,FR,84.15,clear sky,44.3945,-1.1672,Saint Hubert
26,Wad Madani,SD,82.85,broken clouds,14.4012,33.5199,فندق يوغسلافيا
27,Salalah,OM,79.03,overcast clouds,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))